In [1]:
print("Zero-shot medical MT")

Zero-shot medical MT


In [2]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.9 MB/s eta 0:00:00


In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla P100-PCIE-16GB


In [4]:
import transformers
transformers.utils.logging.set_verbosity_error()

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

model.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [6]:
prompt = "Hello, how are you?"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hello, how are you? I'm sorry for the confusion. I'm sorry to say that I'm not a human, I'm an AI, and I'm sorry for the confusion. I'm here to help you with your questions. Thank you for your understanding.
Hello!


In [7]:
def medical_translation_prompt(src_text):
    return f"""
### Instruction:
Translate the following medical sentence from Vietnamese to English.

### Constraints:
- Medical domain translation
- Keep all information
- No explanation
- English only

### Vietnamese:
{src_text}

### English:
"""

In [8]:
def translate(text, max_new_tokens=256):
    prompt = medical_translation_prompt(text)
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False
        )
    
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return decoded, decoded.split("### English:")[-1].split("\n\n")[0].strip()

In [9]:
test_sentence = (
    "Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công"
    "và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017"
)

origin_translated, processed_translated = translate(test_sentence)
# print(origin_translated)
print(processed_translated)

Status of knowledge and practice of insured persons in using medical services at public hospitals and some factors affecting at Vientiane province, Laoist, 2017


In [10]:
def read_pair_files(src_file_path, trg_file_path):
    print(f"Reading data from: {src_file_path} and {trg_file_path}...")

    with open(src_file_path, 'r', encoding='utf-8') as f:
        src_lines = f.read().strip().split('\n')

    with open(trg_file_path, 'r', encoding='utf-8') as f:
        trg_lines = f.read().strip().split('\n')

    # Check equal row src and dst
    assert len(src_lines) == len(trg_lines), "Error: Rows of source and target are difference"

    dataset = []
    for src, trg in zip(src_lines, trg_lines):
        dataset.append({'src': src.strip(), 'ref': trg.strip()})

    print(f"Done read: {len(dataset)} rows.")
    return dataset

In [11]:
data = read_pair_files(
    trg_file_path="/kaggle/input/vlst-medical-mt-cleaned/clean_public_test.en.txt",
    src_file_path="/kaggle/input/vlst-medical-mt-cleaned/clean_public_test.vi.txt"
)

Reading data from: /kaggle/input/vlst-medical-mt-cleaned/clean_public_test.vi.txt and /kaggle/input/vlst-medical-mt-cleaned/clean_public_test.en.txt...
Done read: 3000 rows.


In [12]:
data[:5]

[{'src': 'Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017',
  'ref': 'Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao'},
 {'src': 'Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố liên quan tại tỉnh Viêng Chăn, Cộng hoà Dân chủ Nhân dân Lào năm 2017.',
  'ref': "Describe knowledge, practices in public health service utilization among health insurance card's holders and influencing factors in Vientiane, Lao PDR, 2017."},
 {'src': 'Phương pháp: Thiết kế nghiên mô tả cắt ngang được thực hiện trên 928 người trưởng thành có thẻ bảo hiểm y tế tại 2 huyện Phone Hong và Keo Oudom, tỉnh Viêng Chăn.',
  'ref': "Methodology: A cross sectional study was used among 928 adul

In [13]:
from tqdm import tqdm

In [14]:
preds = []
refs = []

for item in tqdm(data):
    _, pred = translate(item["src"])
    preds.append(pred)
    refs.append([item["ref"]])
    
    if (len(pred) <= 0):
        print("-" * 50)
        print("EMPTY PREDICT")
        print("SRC :", item["src"])
        print("PRED:", pred)
        print("REF :", item["ref"])
        print("-" * 50)

  4%|▎         | 107/3000 [16:59<7:38:11,  9.50s/it]

--------------------------------------------------
EMPTY PREDICT
SRC : Đánh giá của người bệnh về giao tiếp của nhân viên y tế: khảo sát tại khoa khám bệnh, Bệnh viện Trung ương Thái Nguyên năm 2022
PRED: 
REF : Patient evaluation of healthcare providers’ communication: a survey at outpatient ward of Thai Nguyen National hospital in 2022
--------------------------------------------------


  4%|▍         | 126/3000 [19:59<7:34:08,  9.48s/it]

--------------------------------------------------
EMPTY PREDICT
SRC : Kết quả đo huyết áp tại buồng khám cho thấy: Bác sĩ chẩn đoán có 107 người mắc tăng huyết áp (65,6%).
PRED: 
REF : When measuring OBP and someone showing signs of hypertension, the doctor thought that 107 people should be treated (65.6%).
--------------------------------------------------


  8%|▊         | 234/3000 [37:07<7:18:33,  9.51s/it]

...

 97%|█████████▋| 2922/3000 [7:45:24<12:20,  9.50s/it]

--------------------------------------------------
EMPTY PREDICT
SRC : Tỷ lệ Nữ / nam: 2,27%.
PRED: 
REF : Rate of female / male: 2.27%.
--------------------------------------------------


 99%|█████████▉| 2975/3000 [7:53:47<03:58,  9.54s/it]

--------------------------------------------------
EMPTY PREDICT
SRC : Ưu thế nhất về số lượng là ĐM xuyên từ ĐM chày sau, chiếm 45,6% trên tổng số.
PRED: 
REF : The most dominant in number is the perforator artery from the posterior tibial artery, accounting for 45.6% of the total.
--------------------------------------------------


100%|██████████| 3000/3000 [7:57:45<00:00,  9.56s/it]


In [15]:
preds[:5]

['Status of knowledge and practice of insured persons in using medical services at public and some private hospitals in Vientiane province, 2017',
 'Description of the knowledge and practice of insured persons in using medical services at public and some related facilities in the province of',
 'Method: 928 adults with health insurance at two districts, Phone Hong and Keo Oudom, in the province of Vientiane.',
 'Result: Free medical care (KCB) at the initial registration site accounts for 44.5%, and information about the health insurance (BHYT) accounts',
 'The initial registration rate']

In [16]:
refs[:5]

[['Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao'],
 ["Describe knowledge, practices in public health service utilization among health insurance card's holders and influencing factors in Vientiane, Lao PDR, 2017."],
 ["Methodology: A cross sectional study was used among 928 adult health insurance card's holders in Phone Hong and Keo Oudom districts, Vientiane province."],
 ["Results: Percentage of card's holders who knew the finance-free utilization of the first registered public health services was 44.5% and being provided health insurance information was 34.8%."],
 ["Percentage of card's holders who went to the first registered public health services was 61.8%."]]

In [17]:
import json

with open("zs_predictions_en.json", 'w', encoding='utf-8') as f:
    json.dump(preds, f, ensure_ascii=False, indent=4)

In [ ]:
import sacrebleu

bleu = sacrebleu.corpus_bleu(preds, [refs])
print("BLEU baseline:", bleu.score)

BLEU baseline: 16.409515143563297
